# Web scrapeando información sobre equipos de eSports

### Escrito por Jorge Quispe

### Introducción

##### Trabajaremos con la API de PandaScore, una página dedicada a brindar información sobre la escena gaming profesional. Esta posee una interfaz amigable, teniendo así una columna en la izquierda donde nos muestra instrucciones para principiantes y los distintos tipos de solicitudes que podemos hacer donde a medida que seleccionemos uno de estos tendremos una interfaz a la derecha donde aparecerá una plantilla de código que cambiará respecto a la información que añadas y en el lenguaje de programación que desees y por si fuera poco también cuenta con un botón de Modo Oscuro para quienes gustan trabajar así. Es necesario tener una cuenta, una vez te hayas reguistrado lo primero que te aparecerá será tu api key (token), esta tiene mucho tiempo por lo que no es necesario preocuparnos por si esta deja de ser válida. Puedes tener más información y registrarte <a href= https://developers.pandascore.co/>aquí</a>

### Objetivo
##### A partir de la liga coreana (LCK) de eSports de League of Legends (LoL) buscaré toda la información que se me pueda brindar acerca de cada jugador. La idea es poder visualizar cuántos equipos conforman la liga y qué información PandaScore nos brinda sobre cada jugador de este equipo.

Importamos las librerias necesarias para el web scraping, Panda Score devuelve la información en formato JSON.

In [2]:
import requests
import polars as pl
import json

Usando la sección de la columna izquierda antes mencionada (o usando su buscador) que nos interese, generaremos un link de referencia sobre lo que estemos buscando. 

A fin de lograr información actual busqué la serie más reciente de la LCK, la del 2023 que se dió en los primeros meses de este año. Según PandaScore, hallamos esto bajo la id "5954". Procedemos a seguir "armando" nuestro link.

Una vez terminado guardamos el token y la url que generamos y las combinamos.

NOTA: Cada API web tiene una manera para mencionar el argumento de que colocaremos nuestro token, PandaScore pide que esta sea <i>token=</i>, el signo anterior puede variar dependiendo de si es tu primer argumento ( <i>?</i> ) o no ( <i>&</i> ).

In [3]:
api_url = "https://api.pandascore.co/lol/series/5954/teams/"
api_key = "qM4aTj1RzAKr8NTAvP49Jz3RYl9bOdXcCI3VnojkKOJAqIymV_s" #llamarlo como '.token='

url = api_url + '?token=' + api_key
print(url)

https://api.pandascore.co/lol/series/5954/teams/?token=qM4aTj1RzAKr8NTAvP49Jz3RYl9bOdXcCI3VnojkKOJAqIymV_s


Creamos una request solicitando acceso a la url, si la respuesta del servidor es '200' significa que esta fue aceptada. 

In [5]:
req = requests.get(url)

print(req)

<Response [200]>


Convertimos la información en un diccionario para que Python pueda leerlo. Para hacerlo mejor visible lo pasamos a un DataFrame. Visualizaremos mucha información sobre cada equipo de la LCK, que a simple vista notamos que la conforman 8.

In [6]:
resp = req.json()
#print(resp)
df = pl.DataFrame(resp)
print(df)

shape: (10, 9)
┌─────────┬────────────┬────────┬────────────┬───┬───────────┬────────────┬────────────┬───────────┐
│ acronym ┆ current_vi ┆ id     ┆ image_url  ┆ … ┆ modified_ ┆ name       ┆ players    ┆ slug      │
│ ---     ┆ deogame    ┆ ---    ┆ ---        ┆   ┆ at        ┆ ---        ┆ ---        ┆ ---       │
│ str     ┆ ---        ┆ i64    ┆ str        ┆   ┆ ---       ┆ str        ┆ list[struc ┆ str       │
│         ┆ struct[3]  ┆        ┆            ┆   ┆ str       ┆            ┆ t[11]]     ┆           │
╞═════════╪════════════╪════════╪════════════╪═══╪═══════════╪════════════╪════════════╪═══════════╡
│ DK      ┆ {1,"LoL"," ┆ 132531 ┆ https://cd ┆ … ┆ 2023-06-1 ┆ Dplus KIA  ┆ [{26,"1996 ┆ dplus-kia │
│         ┆ league-of- ┆        ┆ n.pandasco ┆   ┆ 7T14:26:2 ┆            ┆ -10-23","K ┆           │
│         ┆ legends"}  ┆        ┆ re.co/imag ┆   ┆ 7Z        ┆            ┆ im",2169," ┆           │
│         ┆            ┆        ┆ es…        ┆   ┆           ┆            ┆ 

Si se desean visualizar mejor el nombre de las columnas podemos usar:

In [7]:
print(df.columns)

['acronym', 'current_videogame', 'id', 'image_url', 'location', 'modified_at', 'name', 'players', 'slug']


Jugaremos con dos columnas de nuestro DataFrame "df", <i>acronym</i> y <i>players</i>. La primera te pone las abreviación del nombre (acrónimo) del equipo  y la segunda información de los jugadores (en una lista de varios elementos). Haremos que la información sea concisa y estética colocando el acrónimo de cada equipo y abajo un DataFrame con la información sobre los jugadores que la conforman. También podíamos haber usado la variable 'name' en lugar de 'acronym' si se deseaba ver el nombre completo de los equipos. <br><br>
El resultado es:

In [27]:
contador = 0
grupo = []
for equipo in df['acronym']:
    print(equipo)
    for info in df['players'][contador]:
        grupo.append(info)
    grupo_df = pl.DataFrame(grupo)
    print(grupo_df)
    #print(grupo_df.columns)
    contador += 1
    grupo.clear()
        

DK
shape: (6, 11)
┌─────┬────────────┬────────────┬───────┬───┬───────────┬─────────────┬──────┬───────────┐
│ age ┆ birthday   ┆ first_name ┆ id    ┆ … ┆ name      ┆ nationality ┆ role ┆ slug      │
│ --- ┆ ---        ┆ ---        ┆ ---   ┆   ┆ ---       ┆ ---         ┆ ---  ┆ ---       │
│ i64 ┆ str        ┆ str        ┆ i64   ┆   ┆ str       ┆ str         ┆ str  ┆ str       │
╞═════╪════════════╪════════════╪═══════╪═══╪═══════════╪═════════════╪══════╪═══════════╡
│ 26  ┆ 1996-10-23 ┆ Kim        ┆ 2169  ┆ … ┆ Deft      ┆ KR          ┆ adc  ┆ deft      │
│ 22  ┆ 2000-07-22 ┆ Heo        ┆ 8166  ┆ … ┆ ShowMaker ┆ KR          ┆ mid  ┆ showmaker │
│ 22  ┆ 2001-06-18 ┆ Kim        ┆ 18112 ┆ … ┆ Canyon    ┆ KR          ┆ jun  ┆ canyon    │
│ 22  ┆ 2001-02-01 ┆ Kim        ┆ 19302 ┆ … ┆ Kellin    ┆ KR          ┆ sup  ┆ kellin    │
│ 23  ┆ 2000-02-11 ┆ Kim        ┆ 24808 ┆ … ┆ Canna     ┆ KR          ┆ top  ┆ canna     │
│ 19  ┆ 2004-02-13 ┆ Cho        ┆ 35024 ┆ … ┆ Rahel     ┆ KR          ┆ 

También podemos precisar esa información solo para un equipo. En este caso seleccionamos al equipo Gen.G cuyo acrónimo es <i>GEN</i>.

In [30]:
contador = 0
gen_g = []
for equipo in df['acronym']:
    if equipo == "GEN":
        print(equipo)
        for nombre in df['players'][contador]:
            gen_g.append(nombre)
    contador += 1

gen_g_df = pl.DataFrame(gen_g)
print(gen_g_df)
print(gen_g_df.columns) #por si queremos visualizar las columnas

GEN
shape: (5, 11)
┌─────┬────────────┬────────────┬───────┬───┬─────────┬─────────────┬──────┬───────────────────────┐
│ age ┆ birthday   ┆ first_name ┆ id    ┆ … ┆ name    ┆ nationality ┆ role ┆ slug                  │
│ --- ┆ ---        ┆ ---        ┆ ---   ┆   ┆ ---     ┆ ---         ┆ ---  ┆ ---                   │
│ i64 ┆ str        ┆ str        ┆ i64   ┆   ┆ str     ┆ str         ┆ str  ┆ str                   │
╞═════╪════════════╪════════════╪═══════╪═══╪═════════╪═════════════╪══════╪═══════════════════════╡
│ 25  ┆ 1998-02-03 ┆ Han        ┆ 520   ┆ … ┆ Peanut  ┆ KR          ┆ jun  ┆ peanut                │
│ 22  ┆ 2001-03-03 ┆ Jeong      ┆ 15000 ┆ … ┆ Chovy   ┆ KR          ┆ mid  ┆ chovy                 │
│ 22  ┆ 2000-07-22 ┆ Choi       ┆ 19282 ┆ … ┆ Doran   ┆ KR          ┆ top  ┆ doran-2686e2c9-bd34-4 │
│     ┆            ┆            ┆       ┆   ┆         ┆             ┆      ┆ 7fb-9d17-87…          │
│ 20  ┆ 2002-09-12 ┆ Yoo        ┆ 31220 ┆ … ┆ Delight ┆ KR          ┆ su

Entonces hemos visto que podemos buscar y ordenar información con las APIs de PandaScore para que esta se nos presente concisa y estética. Así como obtuvimos la información de cada jugador de una liga clasificados por sus respectivos equipos, PandaScore nos ofrece información sobre los resultados de los enfrentamientos y más, para que podamos formular solicitudes de cada tipo, uniendo varias ligas, recogiendo el historial entre dos equipos con su información adicional (asesinatos, número de objetivos realizados, cantidad de oro obtenido, duración aproximada de cada partida) y más. La información ofrecida pueden servir tanto para fines de prática/estudio de estos temas como para llegar incluso a sentirte respaldado a la hora de apostar, eso sí, de optar por esta opción pido encarecidamente ser mayor de edad y jugar de manera responsable.